# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here
import pandas as pd
import cleaning as cl
pd.set_option('display.max_columns', None)
data_df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")

"""
1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.
"""
#print(data_df.columns)

cl.rename_columns(data_df)

data_low_claim_amount_df = data_df[data_df["total_claim_amount"] < 1000]
data_response_yes = data_low_claim_amount_df[data_low_claim_amount_df["response"] == "Yes"]
#print(data_response_yes)
grouped_low_claim_amount = data_response_yes.groupby(["policy_type", "gender"])[["monthly_premium_auto", "customer_lifetime_value"]].mean()
# print(grouped_low_claim_amount)
"""
2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.
"""
data_yes_all_df = data_df[data_df["response"] == "Yes"]
grouped_all = data_df.groupby(["policy_type", "gender"])[["monthly_premium_auto", "customer_lifetime_value"]].mean()

# print(grouped_all)

# most profitable policy tyüe seems to be special auto, because the average customer lifetime value is the highest. 


""" 
3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.
"""
grouped_state = data_df.groupby("state")["customer"].count().reset_index()
state_more_than_500 = grouped_state[grouped_state["customer"]>500]
# print(state_more_than_500)


"""
4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
"""

grouped_gender = data_df.groupby(["gender", "education"])["customer_lifetime_value"].agg(["max", "min", "median"])
# print(grouped_gender)
# the higher the education, the lower the median lifetime value


"""
5. The marketing team wants to analyze the number of policies sold by state and month. 
   Present the data in a table where the months are arranged as columns and the states are arranged as rows.
"""
# print(data_df)

data_df["effective_to_date"] = pd.to_datetime(data_df["effective_to_date"])
data_df["month_nr"] = data_df["effective_to_date"].dt.month
grouped_policy = data_df.groupby(["state", "month_nr", "policy_type"])["customer"].count().reset_index()
pivoted = grouped_policy.pivot(index=["state", "policy_type"], columns="month_nr", values="customer")

#data_df.head()
print(pivoted)




month_nr                      1     2
state      policy_type               
Arizona    Corporate Auto   214   167
           Personal Auto    747   722
           Special Auto      47    40
California Corporate Auto   461   374
           Personal Auto   1391  1203
           Special Auto      66    57
Nevada     Corporate Auto   120    99
           Personal Auto    405   334
           Special Auto      26     9
Oregon     Corporate Auto   305   287
           Personal Auto   1191   989
           Special Auto      69    68
Washington Corporate Auto    86    89
           Personal Auto    363   319
           Special Auto      14    17


C:\Users\Friedrich\AppData\Local\Temp\ipykernel_28944\1105798058.py:57: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_df["effective_to_date"] = pd.to_datetime(data_df["effective_to_date"])


KeyError: 'customers'